### imports

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.spatial import ConvexHull
from matplotlib import ticker, cm
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from IPython.display import clear_output

from ipynb.fs.full.optimizedTIO_paper import refAccuracyScheduler

clear_output()

### dimensions & parameters

In [3]:
myseed = '2404'
np.random.seed(int(myseed))

# basic dimensions
d = int(1e3)
n = int(5e2)
sigma = 1e-3
mu_test = 1e-1
L_test = 2/sigma+mu_test # see Devolder, Glineur, Nesterov 2013 

# TOIO parameters
M = 100

REPS = 10
benchmark = True

# data
O = np.random.normal(0,5,(n,d))
theta_hat = np.mean(O,0)
MAT_O = O@O.T
eigvals = np.real(np.linalg.eigvals(MAT_O))
min_eig,max_eig = max(0,min(eigvals)),max(eigvals)
min_eig/max_eig

0.03177225374474388

### utils

In [4]:
"""
projection on simplex
"""
reltol_simplex,abstol_infeas_simplex = 1e-6,1e-7
def projection_simplex(vector,args=None,inequality=False):
    if args is None:
        radius = 1
    else:
        radius = args[0]
    if inequality and min(vector)>=-abstol_infeas_simplex and max(sum(np.maximum(0,vector))-radius,0)<=reltol_simplex*radius:
        return vector
    n_features = vector.shape[0]
    u = np.sort(vector)[::-1]
    cssv = np.cumsum(u) - radius
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(vector - theta, 0)
    return w

In [5]:
"""
(inexact FOIO) FGD from Stonyakin et. al. (2021)
"""
def FGD(x0,f_oracle,Psi=lambda u: 0, proxPsi = lambda u,g,ss: u, accuracy_schedule=np.zeros(int(1e3)),L=1.0,m=0.0,mu=0.0,verbose=True,freq=10,keep=True):
    
    h_grad = lambda u: u # relative smoothness kernel's gradient
    
    # init.
    k = 0
    x,y,v = x0.copy(),x0.copy(),x0.copy()
    warm_start_pack = None
    F_out = np.inf
    dimension = len(x)
    if type(L)==type([]):
        L_schedule = L.copy()
    else :
        L_schedule = L*np.ones(len(accuracy_schedule))
        
    s = mu+m
    A = 0
    accum_k = 0
    omega_count = 0

    # storing sequence
    infos = np.array([[0,0,0,0]])
    
    # main algorithmic loop
    start_ = time.time()
    
    for id_accuracy,accuracy_k in enumerate(accuracy_schedule):
        
        local_L = L_schedule[id_accuracy] # local model smoothness parameter (update)
        a_plus = ((1+A*s)+((1+A*s+4*A*local_L)**(1/2) * (1+A*s)**(1/2)))/(2*local_L) # new impact
        
        y = (A*x + a_plus*v)/(A+a_plus)
        
        fk,gk,warm_start_pack,true_delta_k,omega_k = f_oracle(y,'full',accuracy=accuracy_k,warm_start=warm_start_pack) # FOI collection
        omega_count += omega_k
        accum_k = (A*accum_k+(A+a_plus)*true_delta_k)/(A+a_plus)
        Fyk = fk+Psi(y)+true_delta_k/2 #upper-bound
        
        time_k = time.time()-start_
        if verbose and (k % freq == 0) and k>0:
            print("k = %i, value = %e, totTime = %e, totError = %e" %
                  (k,Fyk,time_k,accum_k))
            print(' ')
            
        v = proxPsi(h_grad(v)*local_L*a_plus/(A+a_plus) + h_grad(y)*mu, gk, stepsize=(mu+local_L*a_plus/(A+a_plus))**(-1)) # PROXIMAL step 1
        x = (A*x + a_plus*v)/(A+a_plus)
        
        if keep:
                        
            infos = np.vstack((infos,np.array([[Fyk,time_k,accum_k,omega_count]]))) # storage time 

        k += 1 # updates of iteration counter and accumulators
        A += a_plus   

    if keep:            
        
        return x,np.array(infos[1:])
    
    else:
        
        return x,time_k,omega_count

In [6]:
"""
momentum for inner problem 
"""
if min_eig*sigma>0:
    r = 0
    def momentum(theta):
        return 1/(1+2*np.sqrt(min_eig/max_eig)),theta
else:
    r = 1/2
    def momentum(theta):
        new_theta = 1/2*(theta*np.sqrt(theta**2+4)-theta**2)
        inertia = new_theta*(1/theta-1)
        return inertia,new_theta

In [7]:

my_proj = lambda u: projection_simplex(u)

def myPsi(x):
    return 0

def myProxPsi(anchor, direction, stepsize):
    return projection_simplex(-stepsize*(direction-anchor))

def myLMM(anchor,direction,mu=0.0):
    id_min = np.argmin(direction)
    buf = np.zeros(len(anchor))
    if mu==0:
        buf[id_min] = 1
        return buf # for radius 1
    else:
        return projection_simplex(anchor-direction*mu**(-1))

In [8]:
### oracle preparation
def RLP_oracle(x,mode='value',accuracy=1e-3,warm_start=None,max_inner=int(5e4),verb=False):
    
    ## implementation of FISTA with FW-GAP and FO worst-case bound monitoring
    if warm_start is not None:
        inner_seq1,inner_seq1_lag,inner_seq2 = warm_start.copy(),warm_start.copy(),warm_start.copy()
    else:
        inner_seq1,inner_seq1_lag,inner_seq2 = np.ones(n)/n,np.ones(n)/n,np.ones(n)/n
        
    k_inner,cond = 0,False
    
    stepsize = 1/(sigma*max_eig)
    local_strong = sigma*min_eig
 
    lower_bound = -np.inf
    inertia,theta = 0,1
    grad_shift = (x+sigma*theta_hat)@O.T
    
    while k_inner<max_inner and cond==False:
        # proximal gradient step
        inner_seq1_lag = inner_seq1
        grad_inner_seq2 = sigma*MAT_O@inner_seq2-grad_shift
        inner_seq1 = projection_simplex(inner_seq2-stepsize*grad_inner_seq2)
        # momentum 
        inertia,theta = momentum(theta) 
        inner_seq2 = inner_seq1 + inertia*(inner_seq1-inner_seq1_lag)
        # FW check
        grad_inner_seq1 = sigma*MAT_O@inner_seq1-grad_shift
        buf = np.zeros(n)
        buf[np.argmin(grad_inner_seq1)] = 1
        val_inner_seq1 = sigma/2*sum((inner_seq1@O-theta_hat)**2) - sum((inner_seq1@O)*x)
        lower_bound = max(lower_bound,sum(grad_inner_seq1*(buf-inner_seq1))+val_inner_seq1)
        acc_cert = min(4/stepsize/(k_inner+1)**2,2/stepsize*(1-np.sqrt(stepsize*local_strong))**(k_inner),val_inner_seq1-lower_bound)
        cond = acc_cert<=accuracy/2
        if verb and k_inner%10==0:
            print('inner it -> '+str(k_inner+1)+' | DUALITY GAP: '+str(val_inner_seq1-lower_bound))
        # inner iteration counter increase
        k_inner+=1
        
    # evaluation time
    if mode=='value':
        return -val_inner_seq1 + mu_test/2*sum(x**2),inner_seq1,acc_cert*2,k_inner
    elif mode=='gradient':
        return inner_seq1@O + mu_test*x,inner_seq1,acc_cert*2,k_inner
    else:
        return -val_inner_seq1 + mu_test/2*sum(x**2), inner_seq1@O+mu_test*x,inner_seq1,acc_cert*2,k_inner

### `experiment` 

multiple variations of $\bar{\delta}$ and $N_r$ in <i>offline</i> schedules

In [9]:
### creation of dictionaries

import pandas as pd
fields = ['d','dim_O','N_ref','bar_delta','r','GAP','schedule','condition','time','prox_iter']
dic_results = {}
for field in fields:
    dic_results[field] = []

Nr_list = [100,500,1000,5000]
if r==0:
    bar_delta_list = [9e-3,1e-4,1e-6]
else:
    bar_delta_list = [1e-1,1e-3,1e-5]

In [10]:
### benchmark to refine F_star
x0 = my_proj(np.random.uniform(-2,2,d))
if r==0:
    x_benchmark,time_benchmark,_ = FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),Psi = myPsi, proxPsi = myProxPsi, accuracy_schedule=np.ones(8000)*1e-8, L=L_test,m=0.0,mu=mu_test,verbose=True,freq=100,keep=False)
else:
    x_benchmark,time_benchmark,_ = FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),Psi = myPsi, proxPsi = myProxPsi, accuracy_schedule=np.ones(5000)*1e-7, L=L_test,m=0.0,mu=mu_test,verbose=True,freq=10,keep=False)
val_benchmark,grad_benchmark,_,error_benchmark,_ = RLP_oracle(x_benchmark,mode='full',accuracy=1e-10,max_inner=int(5e4),verb=False)
rectified_val_benchmark = val_benchmark-error_benchmark
minimizer_lb_benchmark = myLMM(x_benchmark,grad_benchmark,mu_test)
lb_benchmark = rectified_val_benchmark + sum(grad_benchmark*(minimizer_lb_benchmark-x_benchmark))+ mu_test/2*sum((minimizer_lb_benchmark-x_benchmark)**2)

k = 100, value = -1.850911e-01, totTime = 7.680486e+00, totError = 3.036919e-07
 
k = 200, value = -1.880056e-01, totTime = 1.295212e+01, totError = 5.717581e-07
 
k = 300, value = -1.885438e-01, totTime = 1.712418e+01, totError = 7.791315e-07
 
k = 400, value = -1.887211e-01, totTime = 2.048889e+01, totError = 9.356027e-07
 
k = 500, value = -1.887937e-01, totTime = 2.317421e+01, totError = 1.080311e-06
 
k = 600, value = -1.888265e-01, totTime = 2.543666e+01, totError = 1.132094e-06
 
k = 700, value = -1.888419e-01, totTime = 2.730896e+01, totError = 1.176284e-06
 
k = 800, value = -1.888494e-01, totTime = 2.869285e+01, totError = 1.215119e-06
 
k = 900, value = -1.888530e-01, totTime = 2.968738e+01, totError = 1.224693e-06
 
k = 1000, value = -1.888548e-01, totTime = 3.037812e+01, totError = 1.193762e-06
 
k = 1100, value = -1.888557e-01, totTime = 3.092198e+01, totError = 1.158484e-06
 
k = 1200, value = -1.888561e-01, totTime = 3.135173e+01, totError = 1.137529e-06
 
k = 1300, val

In [11]:
F_star = lb_benchmark

In [12]:
condition = np.format_float_scientific(mu_test/L_test,3)

for N_ref in Nr_list:
    for bar_delta in bar_delta_list:
            
        accuracy_ref = refAccuracyScheduler(N_ref,M,bar_delta,r,arguments=[mu_test,0.0,L_test],typ='FGD',verbose=False)
                
        print('new scenario || N = '+str(N_ref)+', bar_delta = '+str(bar_delta)+', r = '+str(r))
        print(' ')

        for i_rep in range(REPS):
            
            x0 = my_proj(np.random.uniform(0,1,d))

            print('rep#'+str(i_rep))
            print(' ')
            
            # constant phase
            x_constant,time_constant,omega_constant = \
               FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),Psi = myPsi, proxPsi = myProxPsi, accuracy_schedule=np.ones(N_ref)*bar_delta, L=L_test,m=0.0,mu=mu_test,verbose=False,freq=10,keep=False)
            val_constant,grad_constant,_,error_constant,_ = RLP_oracle(x_constant,mode='full',accuracy=1e-10,max_inner=int(5e4),verb=False)
            GAP_constant = val_constant-F_star
            print('constant: '+str(np.format_float_scientific(GAP_constant,3))+' in '+str(time_constant)+' [s]'+' for '+\
                  str(omega_constant)+' inner-iterations')
            # tunable phase
            x_tunable,time_tunable,omega_tunable = \
            FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),Psi = myPsi, proxPsi = myProxPsi, accuracy_schedule=accuracy_ref, L=L_test,m=0.0,mu=mu_test,verbose=False,freq=10,keep=False)
            val_tunable,grad_tunable,_,error_tunable,_ = RLP_oracle(x_tunable,mode='full',accuracy=1e-10,max_inner=int(5e4),verb=False)
            GAP_tunable = val_tunable-F_star
            print('tunable: '+str(np.format_float_scientific(GAP_tunable,3))+' in '+str(time_tunable)+' [s]'+' for '+\
                  str(omega_tunable)+' inner-iterations')
            print(' ')
            dic_results['d'].append([d,d])
            dic_results['dim_O'].append([n,n])
            dic_results['condition'].append([condition,condition])
            dic_results['N_ref'].append([N_ref,N_ref])
            dic_results['bar_delta'].append([bar_delta,bar_delta])
            dic_results['r'].append([r,r])
            dic_results['schedule'].append(['constant','tunable'])
            dic_results['GAP'].append([GAP_constant,GAP_tunable])
            dic_results['time'].append([time_constant,time_tunable])
            dic_results['prox_iter'].append([omega_constant,omega_tunable])

            
        print(' ')
        print(' =========== ')
        print(' ')

new scenario || N = 100, bar_delta = 0.009, r = 0
 
rep#0
 
constant: 3.085e-03 in 0.5262999534606934 [s] for 336 inner-iterations
tunable: 3.045e-03 in 0.3840031623840332 [s] for 246 inner-iterations
 
rep#1
 
constant: 3.080e-03 in 0.5464420318603516 [s] for 342 inner-iterations
tunable: 3.017e-03 in 0.37993669509887695 [s] for 248 inner-iterations
 
rep#2
 
constant: 2.660e-03 in 0.497816801071167 [s] for 329 inner-iterations
tunable: 2.622e-03 in 0.3572580814361572 [s] for 242 inner-iterations
 
rep#3
 
constant: 3.281e-03 in 0.4566762447357178 [s] for 327 inner-iterations
tunable: 3.252e-03 in 0.348405122756958 [s] for 238 inner-iterations
 
rep#4
 
constant: 2.993e-03 in 0.4801928997039795 [s] for 344 inner-iterations
tunable: 2.935e-03 in 0.3830568790435791 [s] for 252 inner-iterations
 
rep#5
 
constant: 3.236e-03 in 0.48673200607299805 [s] for 337 inner-iterations
tunable: 3.187e-03 in 0.3739173412322998 [s] for 247 inner-iterations
 
rep#6
 
constant: 2.833e-03 in 0.507437229

tunable: 5.088e-05 in 9.15252685546875 [s] for 7107 inner-iterations
 
rep#2
 
constant: 4.961e-05 in 9.884232997894287 [s] for 7337 inner-iterations
tunable: 4.961e-05 in 9.122349977493286 [s] for 6919 inner-iterations
 
rep#3
 
constant: 4.999e-05 in 9.22640061378479 [s] for 7183 inner-iterations
tunable: 4.999e-05 in 8.600012063980103 [s] for 6601 inner-iterations
 
rep#4
 
constant: 4.516e-05 in 10.888522863388062 [s] for 7055 inner-iterations
tunable: 4.516e-05 in 9.590725183486938 [s] for 6524 inner-iterations
 
rep#5
 
constant: 4.046e-05 in 10.811582326889038 [s] for 7223 inner-iterations
tunable: 4.046e-05 in 10.366506099700928 [s] for 6681 inner-iterations
 
rep#6
 
constant: 5.661e-05 in 10.301927089691162 [s] for 7212 inner-iterations
tunable: 5.661e-05 in 9.436401128768921 [s] for 6676 inner-iterations
 
rep#7
 
constant: 4.96e-05 in 9.726397275924683 [s] for 7352 inner-iterations
tunable: 4.96e-05 in 9.848076820373535 [s] for 6842 inner-iterations
 
rep#8
 
constant: 4.40

tunable: 9.792e-11 in 17.437456130981445 [s] for 9261 inner-iterations
 
rep#4
 
constant: 1.264e-08 in 20.803443908691406 [s] for 11637 inner-iterations
tunable: 9.78e-11 in 15.782896041870117 [s] for 9285 inner-iterations
 
rep#5
 
constant: 1.581e-08 in 19.405411958694458 [s] for 11627 inner-iterations
tunable: 9.780e-11 in 17.014262914657593 [s] for 9298 inner-iterations
 
rep#6
 
constant: 1.878e-08 in 13.53582215309143 [s] for 11663 inner-iterations
tunable: 9.787e-11 in 10.321960926055908 [s] for 9291 inner-iterations
 
rep#7
 
constant: 1.773e-08 in 12.011312007904053 [s] for 11752 inner-iterations
tunable: 9.782e-11 in 9.283349752426147 [s] for 9315 inner-iterations
 
rep#8
 
constant: 1.156e-08 in 11.575700044631958 [s] for 11750 inner-iterations
tunable: 9.785e-11 in 9.437530279159546 [s] for 9337 inner-iterations
 
rep#9
 
constant: 1.243e-08 in 10.673171043395996 [s] for 11673 inner-iterations
tunable: 9.785e-11 in 9.414316654205322 [s] for 9347 inner-iterations
 
 
 
new 

In [13]:
def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

In [14]:
for field in fields:
    dic_results[field] = flatten_extend(dic_results[field])
df = pd.DataFrame(data=dic_results)
if mu_test==0:
    df.to_csv('2_paper_new_rlp_simplex_no_reg_n_'+str(n)+'_d_'+str(d)+'_r_'+str(r)+'.csv')
else:
    df.to_csv('2_paper_new_rlp_simplex_reg_n_'+str(n)+'_d_'+str(d)+'_r_'+str(r)+'.csv')